## Pulizia & PreProcessing

In [ ]:
# 01_preprocessing.ipynb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# === 1. Caricamento ===
file_path = "../data/Db_Cadute_ICOT.xlsx"
df = pd.read_excel(file_path)
print("✅ Dataset caricato con shape:", df.shape)

# === 2. Standardizzazione nomi colonne ===
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("\n", "_")
    .str.replace("(", "")
    .str.replace(")", "")
    .str.replace("=", "")
    .str.replace(">", "")
    .str.replace("<", "")
    .str.replace("/", "_")
    .str.replace(",", "")
)

# Visualizza nomi standardizzati
print("📋 Colonne dopo cleaning:\n", df.columns.tolist())

# === 3. Rimozione colonne identificative ===
id_cols = ['id', 'cognome', 'nome', 'sede', 'data_di_valutazione']
df.drop(columns=id_cols, inplace=True, errors='ignore')

# === 4. Rimozione colonne con > 40% NaN ===
df = df.loc[:, df.isnull().mean() < 0.4]

# === 5. Eliminazione duplicati (solo per sicurezza) ===
df.drop_duplicates(inplace=True)

# === 6. Analisi missing e imputazione ===
# - imputazione media sulle variabili numeriche
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mean())

# === 7. Codifica variabili categoriche ===
# Individua le categoriche (quelle con valori 1/2, 1/2/3 ecc.)
cat_vars = [
    'sex_m1_f2', 'postural_alteration_1si2no',
    'camptocromico1__torre_di_pisa2_flessione_dorsale3_inclinazione_laterale4',
    'falls_last_year_1si_2no',
    'stipsi_1si_2no', 'iposmia_1si_2no',
    'disturbo_sonno_rem_1si_2no', 'depressione_1si_2no',
    'onset_1early__49_2_middle_50-69_3late_70_'
]

from sklearn.preprocessing import LabelEncoder

encoders = {}
for col in cat_vars:
    if col in df.columns:
        df[col] = df[col].astype(str)
        le = LabelEncoder()
        df[col + '_code'] = le.fit_transform(df[col])
        encoders[col] = le

print("📋 Colonne disponibili:", df.columns.tolist())

# === 8. Costruzione feature set + target ===
target_vars = [
    'total_falls',
    'falls_month_rate',
    'falls_year_rate',
    'fallers_1_nonfaller_0',
    'recurrent_fallers1_nonfallers_occasional0'
]

# Rimuove eventuali duplicati tra feature e target
features = df.drop(columns=target_vars, errors='ignore').columns.tolist()
df_model = df[features + target_vars].copy()

# === 9. Salva dataset clean ===
df_model.to_csv("../data/df_fallrisk_clean.csv", index=False)
print("✅ Dataset pulito salvato con shape:", df_model.shape)

✅ Dataset caricato con shape: (71, 48)
📋 Colonne dopo cleaning:
 ['id', 'cognome', 'nome', 'sede', 'data_di_valutazione', 'hr_v', 'hr_ml', 'hr_ap', '%det_v', '%det_ml', '%det_ap', 'sex_m1_f2', 'age', 'heigth', 'weigth', 'age_onset', 'duration_years', 'onset_1early_49_2_middle_50-69_3late_70_', 'ledd', 'postural_alteration_1si2no', 'camptocromico1_torre_di_pisa2_flessione_dorsale3_inclinazione_laterale4', 'falls_last_year_1si_2no', 'updrs-ii', 'updrs-iii', 'motor_updrs', 'h-y', 'tilt', 'obliquity', 'rotation', 'stance', 'swing', 'double_support', 'single_support', 'stride_length', 'cadence', 'gait_speed', 'stipsi_1si_2no', 'iposmia_1si_2no', 'disturbo_sonno_rem_1si_2no', 'depressione_1si_2no', 'mesi_osservati', 'anni_osservati', 'total_falls', 'cadute_con_frattura', 'falls_month_rate', 'falls_year_rate', 'fallers_1_nonfaller_0', 'recurrent_fallers1_nonfallers_occasional0']
📋 Colonne disponibili: ['hr_v', 'hr_ml', 'hr_ap', '%det_v', '%det_ml', '%det_ap', 'sex_m1_f2', 'age', 'heigth', 'we